In [ ]:
import cv2
import cv2.dnn as dnn
import numpy as np
import matplotlib.pyplot as plt
import time

In [ ]:
import tensorflow as tf

In [ ]:
image = cv2.imread('ELPephants/images/722_Camille I right side_12Dec2014.jpg')
scale = 1/255

In [ ]:
width = image.shape[1]
height = image.shape[0]

In [ ]:
def get_output_layers(net):    
    layer_names = net.getLayerNames()    
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    return output_layers

# function to draw bounding box on the detected object with class name
def draw_bounding_box(img, class_id, confidence, x, y, x_plus_w, y_plus_h):
    # label = str(classes[class_id])
    # color = COLORS[class_id]
    # color = 'blue'
    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), 1, 2)
    # cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

In [ ]:
net = dnn.readNet('../darknet/darknet/cfg/yolov3.cfg', '../darknet/darknet/yolov3.weights')

In [ ]:
blob = dnn.blobFromImage(image, scale, (448,448), (0,0,0), True, crop=False)
net.setInput(blob)

In [ ]:
start = time.time()
outs = net.forward(get_output_layers(net))
print('Total time: {}'.format(time.time() - start))

In [ ]:
conf_threshold = 0.5
nms_threshold = 0.4

In [ ]:
class_ids = []
confidences = []
boxes = []

for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > conf_threshold:
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            x = (center_x - w/2.0)
            y = (center_y - h/2.0)
            class_ids.append(class_id)
            confidences.append(float(confidence))
            boxes.append([x, y, w, h])
            # box = detection[0:4] * np.array([width, height, width, height])
            #boxes.append(box.astype(int))

In [ ]:
class_ids

In [ ]:
indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
bboxes = []

test = image
for i in indices:
    i = i[0]
    box = boxes[i]
    x = round(box[0])
    y = round(box[1])
    w = box[2]
    h = box[3]

    draw_bounding_box(test, class_ids[i], confidences[i], round(x), round(y), round(x+w), round(y+h))
    bboxes.append(image[y:y+h, x:w+x])

In [ ]:
plt.imshow(test)

In [ ]:
fig, axes = plt.subplots(1, len(bboxes))

if len(bboxes) == 1:
    axes.imshow(bboxes[0])
else:
    for i in range(len(bboxes)):
        axes[i].imshow(bboxes[i])

In [ ]:
# plt.imshow(bboxes[0])

In [ ]:
test_img = bboxes[0]

In [ ]:
test_blob = dnn.blobFromImage(test_img, scale, (416,416), (0,0,0), swapRB=True, crop=False)
net.setInput(test_blob)

In [ ]:
layer_names = net.getLayerNames()
# layer_names

In [ ]:
test_outs = net.forward('relu_2')
print(test_outs.shape)
out = test_outs[0]
rows = round((out.shape[0] ** (1/2)))
columns = rows
print(rows, columns)
fig,axes = plt.subplots(rows,columns, figsize=(15,15))
for i in range(rows):
    for j in range(columns):
        if i*rows + j < out.shape[0]:
            axes[i][j].set_xticklabels([])
            axes[i][j].set_yticklabels([])
            axes[i][j].imshow(out[i*rows +j], cmap='gray')


In [ ]:
# layersIds, inLayersShapes, outLayersShapes = net.getLayersShapes(test_blob.shape)

In [ ]:
# outLayersShapes[net.getLayerId('relu_1')]